## 构建词汇表
- 给定文本 "The car is blue."，我们可以根据其中出现的单词构建词汇表。  
- 假设嵌入维度为4

In [40]:
# 定义文本
text = "The car is blue."

# 分词处理
tokens = text.split()

# 构建词汇表
vocab = ['[PAD]', '[UNK]', '[CLS]', '[SEP]']  # 特殊标记
for token in tokens:
    if token not in vocab:
        vocab.append(token)

# 输出词汇表
print("Vocabulary:")
# 将词汇表保存为文件
with open("vocab.txt", "w", encoding="utf-8") as f:
    for word in vocab:
        f.write(word + "\n")


Vocabulary:


# 根据词汇表和embedding 维度初始化embedding 矩阵

torch.nn.Embedding  用于创建词嵌入层（embedding layer）。词嵌入层的作用是将离散的词语或符号映射为密集的词嵌入向量，从而实现词语之间的语义关系的表达和学习。
torch.nn.Embedding 类根据指定的词汇表大小和词嵌入维度，创建一个可学习的词嵌入矩阵。
在该矩阵中，每一行表示词汇表中的一个单词，每一列表示词嵌入向量的一个维度。
通过索引或标识符（通常是整数）来访问矩阵中的某一行，从而获取对应单词的词嵌入向量。
当调用 torch.nn.Embedding 类创建词嵌入层时，PyTorch 会自动为该层的词嵌入矩阵进行随机初始化。
在模型的训练过程中，词嵌入矩阵的参数会被不断调整，以使得模型学习到合适的词嵌入表示，从而在特定任务中取得更好的性能。

In [41]:
import torch
# 读取词汇表
vocab = []
with open("vocab.txt", "r", encoding="utf-8") as f:
    for line in f:
        vocab.append(line.strip())

# 输出词汇表
# print("Vocabulary:")
# for word in vocab:
#     print(word)


# 使用torch随机初始化词向量矩阵，词嵌入矩阵是一个矩阵，其行数等于词汇表的大小，而列数则表示词嵌入的维度。每一行对应词汇表中的一个单词
vocab_size = len(vocab)
embedding_dim = 4
embedding = torch.nn.Embedding(vocab_size, embedding_dim)

# 输出词向量矩阵
print("Embedding Matrix:")
print(embedding.weight)

# 获取单词的词嵌入向量
# embedding(torch.tensor(vocab.index('The')))
# embedding(torch.tensor(vocab.index('car')))
# embedding(torch.tensor(vocab.index('is')))
# embedding(torch.tensor(vocab.index('blue')))



# embedding(torch.tensor(5))  # tensor([-1.8072,  0.1537,  0.4305, -0.8552], grad_fn=<EmbeddingBackward0>)


Embedding Matrix:
Parameter containing:
tensor([[ 1.5679e+00, -8.2977e-01,  2.2722e-01,  1.4347e+00],
        [-2.5464e-01, -4.4918e-01,  1.5074e+00, -1.1345e-01],
        [-2.4509e+00, -8.4751e-01, -1.8819e-01, -1.2093e-01],
        [ 1.1403e+00,  1.9157e+00, -5.0030e-01,  2.3038e+00],
        [-1.5727e+00,  1.8791e+00,  1.4679e-01, -6.2875e-01],
        [ 2.4188e-01, -2.0831e-01,  3.9061e-01, -5.5351e-01],
        [-3.8087e-01, -9.3290e-01, -1.8364e-03,  9.9481e-01],
        [ 4.8886e-01,  5.5294e-02,  1.8746e-01,  3.1746e-01]],
       requires_grad=True)


In [42]:
# 获取单词的词嵌入向量
word_indices = [vocab.index(word) for word in ['The', 'car', 'is', 'blue.']]
embeddings = embedding(torch.tensor(word_indices))
embeddings

tensor([[-1.5727e+00,  1.8791e+00,  1.4679e-01, -6.2875e-01],
        [ 2.4188e-01, -2.0831e-01,  3.9061e-01, -5.5351e-01],
        [-3.8087e-01, -9.3290e-01, -1.8364e-03,  9.9481e-01],
        [ 4.8886e-01,  5.5294e-02,  1.8746e-01,  3.1746e-01]],
       grad_fn=<EmbeddingBackward0>)

# 得到Attention 分数

In [36]:
# 定义查询向量、键向量和数值向量的线性层
linear_q = torch.nn.Linear(embedding_dim, embedding_dim)
linear_k = torch.nn.Linear(embedding_dim, embedding_dim)
linear_v = torch.nn.Linear(embedding_dim, embedding_dim)

# 对词嵌入向量进行线性变换，得到查询向量、键向量和数值向量
query_vector = linear_q(embeddings)
key_vector = linear_k(embeddings)
value_vector = linear_v(embeddings)

print("Query vector:", query_vector)
print("Key vector:", key_vector)
print("Value vector:", value_vector)

# 计算注意力分数
attention_score = torch.matmul(query_vector, key_vector.transpose(0, 1))  # 点积操作
print("Attention score:", attention_score)


Query vector: tensor([[-0.6560, -1.2501,  0.0042,  1.1364],
        [-0.7753,  0.3846, -0.1882, -0.1914],
        [-0.5610, -0.2089, -0.1189,  0.1102],
        [ 0.4652,  0.6619, -0.2724, -0.8579]], grad_fn=<AddmmBackward0>)
Key vector: tensor([[-0.0963, -0.5572, -0.2466,  0.3354],
        [ 0.2894,  1.1900,  0.4358, -0.7489],
        [ 0.2498,  0.5448,  0.1842, -0.2809],
        [ 1.0372,  0.6105,  0.6116, -0.6512]], grad_fn=<AddmmBackward0>)
Value vector: tensor([[-0.3098, -0.2229, -0.4055,  0.6239],
        [ 1.2078,  0.2054, -0.2943, -0.6823],
        [ 0.6738, -0.0613, -0.1500, -0.3225],
        [ 0.9124, -0.2617,  0.9845, -1.3736]], grad_fn=<AddmmBackward0>)
Attention score: tensor([[ 1.1399, -2.5268, -1.1633, -2.1810],
        [-0.1575,  0.2947,  0.0349, -0.5597],
        [ 0.2367, -0.5453, -0.3068, -0.8539],
        [-0.6342,  1.4461,  0.6676,  1.2786]], grad_fn=<MmBackward0>)


In [21]:
key_vector

tensor([[-0.1629,  0.6873, -0.6069, -0.3131],
        [-0.8518, -0.9882, -0.5707,  0.2048],
        [ 0.6326, -1.2686,  0.3509, -0.4068],
        [-0.0899,  0.1633, -0.4199, -0.5899]], grad_fn=<AddmmBackward0>)

# 计算The 的四个Attention 分数

In [23]:
import torch

# 假设 "The" 的索引为 index_the
index_the = vocab.index('The')

# 获取 "The" 的词嵌入向量
embedding_the = embedding(torch.tensor(index_the)).unsqueeze(0)  # 添加一个维度以匹配其他词的形状

# 对 "The" 的词嵌入向量进行线性变换，得到查询向量和键向量
query_vector_the = linear_q(embedding_the)
key_vectors = linear_k(embeddings)

# 计算 "The" 对其他几个词的注意力分数
attention_scores = torch.matmul(query_vector_the, key_vectors.transpose(0, 1))

print("Attention scores for 'The':", attention_scores.squeeze())


Attention scores for 'The': tensor([-0.0775,  0.0403, -0.3928, -0.1669], grad_fn=<SqueezeBackward0>)


# 计算 "The" 对自身的注意力分数

In [25]:
import torch

# 假设 "The" 的索引为 index_the
index_the = vocab.index('The')

# 获取 "The" 的词嵌入向量
embedding_the = embedding(torch.tensor(index_the)).unsqueeze(0)  # 添加一个维度以匹配其他词的形状

# 对 "The" 的词嵌入向量进行线性变换，得到查询向量和键向量
query_vector_the = linear_q(embedding_the)
key_vector_the = linear_k(embedding_the)

# 计算 "The" 对自身的注意力分数
self_attention_score = torch.matmul(query_vector_the, key_vector_the.transpose(0, 1))

print("Self-attention score for 'The':", self_attention_score.squeeze())


Self-attention score for 'The': tensor(-0.0775, grad_fn=<SqueezeBackward0>)


# 缩放与softmax

In [46]:
import torch
# 读取词汇表
vocab = []
with open("vocab.txt", "r", encoding="utf-8") as f:
    for line in f:
        vocab.append(line.strip())

vocab_size = len(vocab)
embedding_dim = 4
embedding = torch.nn.Embedding(vocab_size, embedding_dim)


# 获取单词的词嵌入向量
word_indices = [vocab.index(word) for word in ['The', 'car', 'is', 'blue.']]
embeddings = embedding(torch.tensor(word_indices))
embeddings

# 定义查询向量、键向量和数值向量的线性层
linear_q = torch.nn.Linear(embedding_dim, embedding_dim)
linear_k = torch.nn.Linear(embedding_dim, embedding_dim)
linear_v = torch.nn.Linear(embedding_dim, embedding_dim)

# 对词嵌入向量进行线性变换，得到查询向量、键向量和数值向量
query_vector = linear_q(embeddings)
key_vector = linear_k(embeddings)
value_vector = linear_v(embeddings)

print("Query vector:", query_vector)
print("Key vector:", key_vector)
print("Value vector:", value_vector)

# 计算注意力分数
attention_score = torch.matmul(query_vector, key_vector.transpose(0, 1))  # 点积操作
print("Attention score:", attention_score)



Query vector: tensor([[ 0.9569,  0.4605, -0.3294,  0.8474],
        [-0.0338,  0.8051,  0.0091,  0.3534],
        [-1.9663,  0.3242, -0.6665, -0.9690],
        [ 0.4829,  0.2081, -0.5480,  0.3898]], grad_fn=<AddmmBackward0>)
Key vector: tensor([[-0.2928, -0.1069,  0.1948,  0.0379],
        [ 0.4548, -1.0273, -0.4844,  0.0272],
        [ 0.9010,  0.2535,  0.6154,  0.0961],
        [-1.2380,  1.2561, -0.1251,  0.5132]], grad_fn=<AddmmBackward0>)
Value vector: tensor([[ 0.5278,  0.0791,  0.5377, -0.4503],
        [ 0.6857, -0.4299,  0.3223,  0.1451],
        [-0.9255,  1.1365, -2.0049,  0.2499],
        [ 0.3056,  0.4092, -0.0480, -0.4627]], grad_fn=<AddmmBackward0>)
Attention score: tensor([[-0.3615,  0.1448,  0.8577, -0.1302],
        [-0.0610, -0.8372,  0.2132,  1.2334],
        [ 0.3746, -0.9309, -2.1929,  2.4277],
        [-0.2556,  0.2820,  0.1880, -0.0679]], grad_fn=<MmBackward0>)


In [47]:
import torch.nn.functional as F

# 缩放注意力分数
scaled_attention_score = attention_score / torch.sqrt(torch.tensor(embedding_dim, dtype=torch.float32))

# 对注意力分数进行 softmax 操作
attention_weights = F.softmax(scaled_attention_score, dim=-1)

print("Scaled attention score:", scaled_attention_score)
print("Attention weights:", attention_weights)


Scaled attention score: tensor([[-0.1807,  0.0724,  0.4288, -0.0651],
        [-0.0305, -0.4186,  0.1066,  0.6167],
        [ 0.1873, -0.4654, -1.0964,  1.2138],
        [-0.1278,  0.1410,  0.0940, -0.0340]], grad_fn=<DivBackward0>)
Attention weights: tensor([[0.1905, 0.2453, 0.3504, 0.2138],
        [0.2112, 0.1432, 0.2422, 0.4034],
        [0.2179, 0.1135, 0.0604, 0.6083],
        [0.2148, 0.2811, 0.2682, 0.2360]], grad_fn=<SoftmaxBackward0>)
